In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import chart_studio.plotly as ply
from scipy.signal import spectrogram
from utilities.signal_analysis_methods import autocorrelation, autocorrelation_slow

In [1]:
path = 'data/mag_data.csv'
data = pd.read_csv(path)

times = data['Times (s)'].to_numpy()
ch1_voltages = data[' Ch 1 Voltage (V)'].to_numpy()
ch2_voltages = data[' Ch 2 Voltage (V)'].to_numpy()
sample_rate = int(2e3)

ch1_AC_voltages = ch1_voltages-np.mean(ch1_voltages)
ch2_AC_voltages = ch2_voltages-np.mean(ch2_voltages)

NameError: name 'pd' is not defined

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= times,
    y= ch1_AC_voltages, 
    mode='lines', 
    name='Channel 1',
    line=dict(color='rebeccapurple', width=2)
))

fig.add_trace(go.Scatter(
    x= times,
    y= ch2_AC_voltages, 
    mode='lines', 
    name='Channel 2',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Secondary Coil Voltages',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified'
)

In [ ]:
# RMS voltages calculated in one period bins
bin_size = sample_rate
ch1_rms_voltages = [np.sqrt(np.mean(np.square(ch1_AC_voltages[i*bin_size:(i+1)*bin_size]))).tolist() for i in range(30)]
ch2_rms_voltages = [np.sqrt(np.mean(np.square(ch2_AC_voltages[i*bin_size:(i+1)*bin_size]))).tolist() for i in range(30)]

fig = go.Figure()
fig.add_trace(go.Scatter(
    y= ch1_rms_voltages, 
    mode='lines', 
    name='Channel 1',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    y= ch2_rms_voltages, 
    mode='lines', 
    name='Channel 2',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Binned RMS Averages of Secondary Coil Voltages',
    xaxis_title='Time (s)',
    yaxis_title='RMS Voltage (V)',
    template='plotly_dark',
    hovermode='x unified'
)

In [ ]:

ch1_fft = np.fft.rfft(ch1_AC_voltages)
ch1_fftfreq = np.fft.rfftfreq(len(ch1_AC_voltages),1./sample_rate)

ch2_fft = np.fft.rfft(ch2_AC_voltages)
ch2_fftfreq = np.fft.rfftfreq(len(ch2_AC_voltages),1./sample_rate)

fig = go.Figure(data=go.Scatter(
    x= ch1_fftfreq,
    y= np.log(np.abs(ch1_fft)), 
    mode='lines', 
    name='Channel 1',
    line=dict(color='rebeccapurple', width=2)
))
fig.update_layout(
    title='Secondary Coil Frequency Spectrum (Channel 1)',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Power',
    template='plotly_dark',
    hovermode='x unified'
)
fig.show()

fig = go.Figure(data=go.Scatter(
    x= ch2_fftfreq,
    y= np.log(np.abs(ch2_fft)), 
    mode='lines', 
    name='Channel 2',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Secondary Coil Frequency Spectrum (Channel 2)',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Power',
    template='plotly_dark',
    hovermode='x unified'
)
fig.show()

In [ ]:
# Autocorrelations computed over 3 periods
lag_num_steps = 10
ch1_autocorrelations = autocorrelation(ch1_AC_voltages[:sample_rate*3], lag_interval=lag_num_steps)
ch2_autocorrelations = autocorrelation(ch2_AC_voltages[:sample_rate*3], lag_interval=lag_num_steps)

fig = go.Figure()
fig.add_trace(go.Scatter(
    y= ch1_autocorrelations, 
    mode='lines', 
    name='Channel 1',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    y= ch2_autocorrelations, 
    mode='lines', 
    name='Channel 2',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Autocorrelations of Secondary Coil Voltages',
    xaxis_title='Time Lag (steps)',
    yaxis_title='Autocorrelation',
    template='plotly_dark',
    hovermode='x unified'
)
fig.show()

In [ ]:
frequencies, times, Sxx = spectrogram(ch1_AC_voltages, fs=sample_rate, nperseg=1024)

fig = go.Figure(data=go.Heatmap(
    z=10 * np.log10(Sxx[:32]),  # Convert to dB scale
    x=times,
    y=frequencies[:32],
    colorscale='Plasma'
))
fig.update_layout(
    title='Channel 1 Spectrogram',
    xaxis_title='Time [s]',
    yaxis_title='Frequency [Hz]',
    coloraxis_colorbar=dict(title='Power (dB)'),
    template='plotly_dark'
)
fig.show()

frequencies, times, Sxx = spectrogram(ch2_AC_voltages, fs=sample_rate, nperseg=1024)

fig = go.Figure(data=go.Heatmap(
    z=10 * np.log10(Sxx[:32]),  # Convert to dB scale
    x=times,
    y=frequencies[:32],
    colorscale='Plasma'
))
fig.update_layout(
    title='Channel 2 Spectrogram',
    xaxis_title='Time [s]',
    yaxis_title='Frequency [Hz]',
    coloraxis_colorbar=dict(title='Power (dB)'),
    template='plotly_dark'
)
fig.show()